In [145]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [146]:
#import and view dataset 
df = pd.read_csv('NEW_YORK_Offenses.csv')
df.head(10)

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,"1,861",0,0.000,nan,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,"2,577",3,0.000,nan,0,0,3,24,3,20,1,0.000
2,Akron Village,"2,846",3,0.000,nan,0,0,3,16,1,15,0,0.000
3,Albany,"97,956",791,8.000,nan,30,227,526,"4,090",705,"3,243",142,nan
4,Albion Village,"6,388",23,0.000,nan,3,4,16,223,53,165,5,nan
5,Alfred Village,"4,089",5,0.000,nan,0,3,2,46,10,36,0,nan
6,Allegany Village,"1,781",3,0.000,nan,0,0,3,10,0,10,0,0.000
7,Amherst Town,"118,296",107,1.000,nan,7,31,68,"2,118",204,"1,882",32,3.000
8,Amityville Village,"9,519",9,0.000,nan,2,4,3,210,16,188,6,1.000
9,Amsterdam,"18,182",30,0.000,nan,0,12,18,405,99,291,15,0.000


In [147]:
df.columns

Index(['City', 'Population', 'Violent\ncrime',
       'Murder and\nnonnegligent\nmanslaughter',
       'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2',
       'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
       'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3'],
      dtype='object')

#### Propertycrime=α+Population+Population2+Murder+Robbery

In [148]:
#drop columns and rename as needed per above formula 
df = df.drop(['City', 'Violent\ncrime', 'Rape\n(revised\ndefinition)1',  'Rape\n(legacy\ndefinition)2', 'Aggravated\nassault', 'Property\ncrime', 'Burglary', 'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3'] , axis=1)
df.rename(columns={'Population': 'population', 'Murder and\nnonnegligent\nmanslaughter':'murder', 'Robbery': 'robbery'}, inplace=True)
df.head(10)

,population,murder,robbery
0,"1,861",0.000,0
1,"2,577",0.000,0
2,"2,846",0.000,0
3,"97,956",8.000,227
4,"6,388",0.000,4
5,"4,089",0.000,3
6,"1,781",0.000,0
7,"118,296",1.000,31
8,"9,519",0.000,4
9,"18,182",0.000,12


In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 3 columns):
population    348 non-null object
murder        348 non-null float64
robbery       348 non-null object
dtypes: float64(1), object(2)
memory usage: 5.5+ KB


In [150]:
#drop null values and return int values for columns 
df.replace('NaN', np.nan)
df = df.dropna() 
df['population'] = df.population.str.extract('(\d+)', expand=True).astype(int)
df['robbery'] = df.robbery.str.extract('(\d+)', expand=True).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348 entries, 0 to 347
Data columns (total 3 columns):
population    348 non-null int32
murder        348 non-null float64
robbery       348 non-null int32
dtypes: float64(1), int32(2)
memory usage: 8.2 KB


In [151]:
#create column
df['population2'] = np.power(df['population'], 2)
df.head(10)

,population,murder,robbery,population2
0,1,0.000,0,1
1,2,0.000,0,4
2,2,0.000,0,4
3,97,8.000,227,9409
4,6,0.000,4,36
5,4,0.000,3,16
6,1,0.000,0,1
7,118,1.000,31,13924
8,9,0.000,4,81
9,18,0.000,12,324


In [152]:
#create the three categorical variables 
df['murder'] = np.where((df['murder'] > 0), 1, 0) 
df['robbery'] = np.where((df['robbery'] > 0), 1, 0)  
df['population2'] = np.where((df['population2'] > 0), 1, 0) 
df.head(10)

,population,murder,robbery,population2
0,1,0,0,1
1,2,0,0,1
2,2,0,0,1
3,97,1,1,1
4,6,0,1,1
5,4,0,1,1
6,1,0,0,1
7,118,1,1,1
8,9,0,1,1
9,18,0,1,1
